In [6]:
# Imports
%reload_ext autoreload
%autoreload 2 
import os
from embed_time.splitter_static import DatasetSplitter
from embed_time.dataset_static import ZarrCellDataset
from embed_time.dataloader_static import collate_wrapper
from embed_time.model import Encoder, Decoder, VAE
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

### Split data

In [2]:
# Usage example:
parent_dir = '/mnt/efs/dlmbl/S-md/'
output_file = '/home/S-md/embed_time/notebooks/splits/example_split.csv'
train_ratio = 0.7
val_ratio = 0.15
num_workers = -1

# Create the dataset split CSV file
DatasetSplitter(parent_dir, output_file, train_ratio, val_ratio, num_workers).generate_split()

Processing 2 genes...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Combining results...
Creating DataFrame and saving CSV...
Dataset split CSV saved to /home/S-md/embed_time/notebooks/splits/example_split.csv


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    2.2s finished


### Generate dataset

In [3]:
# Usage example:
parent_dir = '/mnt/efs/dlmbl/S-md/'
csv_file = '/home/S-md/embed_time/notebooks/splits/example_split.csv'
split = 'train'
channels = [0, 1, 2, 3]
cell_cycle_stages = 'interphase'
transform = "masks"
crop_size = 150

# Create the dataset
dataset = ZarrCellDataset(parent_dir, csv_file, split, channels, transform, crop_size)

# Print the number of images and shapes of the data
print(f"The dataset contains {len(dataset)} images.")

The dataset contains 13605 images.


### Generate dataloader

In [7]:
# Define the metadata keys
metadata_keys = ['gene', 'barcode', 'stage']
images_keys = ['original_image', 'cell_image']

# Create a DataLoader for the dataset
dataloader = DataLoader(
    dataset, 
    batch_size=16, 
    shuffle=True, 
    collate_fn=collate_wrapper(metadata_keys, images_keys)
)

# Get a batch of data
for i, batch in enumerate(dataloader):
    if i == 0:
        break

In [9]:
dataloader

### Read in model

In [10]:
# Initiate Encoder and Decoder
encoder = Encoder(input_shape=(150, 150, 4),
                  x_dim=4,
                  h_dim1=512,
                  h_dim2=256,
                  z_dim=32)
decoder = Decoder(z_dim=32,
                  h_dim1=256,
                  h_dim2=512,
                  x_dim=4,
                  output_shape=(150, 150))

# Initiate VAE
vae = VAE(encoder, decoder)

model_graph = torchview.draw_graph(vae,  
                                   dataloader,
                                   roll = True,
                                   depth = 3,
                                   device = 'cpu')

# # Train the VAE
# vae.train(dataloader, epochs=10)

NameError: name 'torchview' is not defined

In [ ]:
# 